# 6. How to use the spaCy Matcher

In [9]:
import spacy

In [10]:
from spacy.matcher import Matcher

In [11]:
nlp = spacy.load("en_core_web_sm")

## 6.1. Basic Example

In [12]:
matcher = Matcher(nlp.vocab)
pattern = [
    {"LIKE_EMAIL": True}
]
matcher.add("EMAIL_ADDRESS", [pattern])

In [13]:
doc = nlp("This is an email address: wnattingy@ail.com")
matches = matcher(doc)

In [14]:
print(matches)

[(16571425990740197027, 6, 7)]


In [15]:
print(nlp.vocab[matches[0][0]].text)

EMAIL_ADDRESS


## 6.2. Attributes Taken by Matcher

* ORTH - The exact verbatim of a token (str)

* TEXT - The exact verbatim of a token (str)

* LOWER - The lowercase form of the token text (str)

* LENGTH - The length of the token text (int)

* IS_ALPHA

* IS_ASCII

* IS_DIGIT

* IS_LOWER

* IS_UPPER

* IS_TITLE

* IS_PUNCT

* IS_SPACE

* IS_STOP

* IS_SENT_START

* LIKE_NUM

* LIKE_URL

* LIKE_EMAIL

* SPACY

* POS

* TAG

* MORPH

* DEP

* LEMMA

* SHAPE

* ENT_TYPE

* _ - Custom extension attributes (Dict[str, Any])

* OP

## 6.3. Applied Matcher

In [16]:
with open("/content/wiki_mlk.txt", "r") as f:
  text = f.read()

In [17]:
print(text)

Martin Luther King Jr. (born Michael King Jr.; January 15, 1929 – April 4, 1968) was an American Baptist minister and activist who became the most visible spokesman and leader in the American civil rights movement from 1955 until his assassination in 1968. King advanced civil rights through nonviolence and civil disobedience, inspired by his Christian beliefs and the nonviolent activism of Mahatma Gandhi. He was the son of early civil rights activist and minister Martin Luther King Sr.

King participated in and led marches for blacks' right to vote, desegregation, labor rights, and other basic civil rights.[1] King led the 1955 Montgomery bus boycott and later became the first president of the Southern Christian Leadership Conference (SCLC). As president of the SCLC, he led the unsuccessful Albany Movement in Albany, Georgia, and helped organize some of the nonviolent 1963 protests in Birmingham, Alabama. King helped organize the 1963 March on Washington, where he delivered his famous 

In [18]:
nlp = spacy.load("en_core_web_sm")

## 6.4. Grabbing all Proper Nouns

In [20]:
matcher = Matcher(nlp.vocab)
pattern = [
    {"POS": "PROPN"}
]
matcher.add("PROPER_NOUN", [pattern])
doc = nlp(text)
matches = matcher(doc)
print(len(matches))
for match in matches[:10]:
  print(match, doc[match[1]:match[2]])

102
(451313080118390996, 0, 1) Martin
(451313080118390996, 1, 2) Luther
(451313080118390996, 2, 3) King
(451313080118390996, 3, 4) Jr.
(451313080118390996, 6, 7) Michael
(451313080118390996, 7, 8) King
(451313080118390996, 8, 9) Jr.
(451313080118390996, 10, 11) January
(451313080118390996, 15, 16) April
(451313080118390996, 23, 24) Baptist


### 6.4.1. Improving it with Multi-Word Tokens

In [21]:
matcher = Matcher(nlp.vocab)
pattern = [
    {"POS": "PROPN", "OP": "+"}
]
matcher.add("PROPER_NOUN", [pattern])
doc = nlp(text)
matches = matcher(doc)
print(len(matches))
for match in matches[:10]:
  print(match, doc[match[1]:match[2]])

175
(451313080118390996, 0, 1) Martin
(451313080118390996, 0, 2) Martin Luther
(451313080118390996, 1, 2) Luther
(451313080118390996, 0, 3) Martin Luther King
(451313080118390996, 1, 3) Luther King
(451313080118390996, 2, 3) King
(451313080118390996, 0, 4) Martin Luther King Jr.
(451313080118390996, 1, 4) Luther King Jr.
(451313080118390996, 2, 4) King Jr.
(451313080118390996, 3, 4) Jr.


### 6.4.2. Greedy Keyword Argument

In [22]:
matcher = Matcher(nlp.vocab)
pattern = [
    {"POS": "PROPN", "OP": "+"}
]
matcher.add("PROPER_NOUN", [pattern], greedy="LONGEST")
doc = nlp(text)
matches = matcher(doc)
print(len(matches))
for match in matches[:10]:
  print(match, doc[match[1]:match[2]])

61
(451313080118390996, 83, 88) Martin Luther King Sr.
(451313080118390996, 469, 474) Martin Luther King Jr. Day
(451313080118390996, 536, 541) Martin Luther King Jr. Memorial
(451313080118390996, 0, 4) Martin Luther King Jr.
(451313080118390996, 128, 132) Southern Christian Leadership Conference
(451313080118390996, 247, 251) Director J. Edgar Hoover
(451313080118390996, 6, 9) Michael King Jr.
(451313080118390996, 325, 328) Nobel Peace Prize
(451313080118390996, 422, 425) James Earl Ray
(451313080118390996, 463, 466) Congressional Gold Medal


### 6.4.3. Sorting it to Apperance¶

In [ ]:
matcher = Matcher(nlp.vocab)
pattern = [{"POS": "PROPN", "OP": "+"}]
matcher.add("PROPER_NOUNS", [pattern], greedy='LONGEST')
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])
print (len(matches))
for match in matches[:10]:
    print (match, doc[match[1]:match[2]])

### 6.4.4. Adding in Sequences

In [25]:
matcher = Matcher(nlp.vocab)
pattern = [
    {"POS": "PROPN", "OP": "+"},
    {"POS": "VERB"}
]
matcher.add("PROPER_NOUN", [pattern], greedy="LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])
print(len(matches))
for match in matches[:20]:
  print(match, doc[match[1]:match[2]])

7
(451313080118390996, 49, 51) King advanced
(451313080118390996, 89, 91) King participated
(451313080118390996, 113, 115) King led
(451313080118390996, 167, 169) King helped
(451313080118390996, 247, 252) Director J. Edgar Hoover considered
(451313080118390996, 322, 324) King won
(451313080118390996, 485, 488) United States beginning


## 6.5. Finding Quotes and Speakers

In [26]:
import json
with open ("/content/alice.json", "r") as f:
  data = json.load(f)

In [28]:
text = data[0][2][0]
print(text)

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'


In [29]:
text = text.replace("`", "'")
print(text)

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'


In [30]:
matcher = Matcher(nlp.vocab)
pattern = [
    {"ORTH": "'"},
    {"IS_ALPHA": True, "OP": "+"},
    {"IS_PUNCT": True, "OP": "*"},
    {"ORTH": "'"}
]
matcher.add("PROPER_NOUN", [pattern], greedy="LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])
print(len(matches))
for match in matches[:20]:
  print(match, doc[match[1]:match[2]])

2
(451313080118390996, 47, 58) 'and what is the use of a book,'
(451313080118390996, 60, 67) 'without pictures or conversation?'


### 6.5.1. Find Speaker

In [33]:
speak_lemmas = ["think", "say"]
matcher = Matcher(nlp.vocab)
pattern = [
    {"ORTH": "'"},
    {"IS_ALPHA": True, "OP": "+"},
    {"IS_PUNCT": True, "OP": "*"},
    {"ORTH": "'"},
    {"POS": "VERB", "LEMMA": {"IN": speak_lemmas}},
    {"POS": "PROPN", "OP": "+"},
    {"ORTH": "'"},
    {"IS_ALPHA": True, "OP": "+"},
    {"IS_PUNCT": True, "OP": "*"},
    {"ORTH": "'"}
]
matcher.add("PROPER_NOUN", [pattern], greedy="LONGEST")
doc = nlp(text)
matches = matcher(doc)
matches.sort(key = lambda x: x[1])
print(len(matches))
for match in matches[:20]:
  print(match, doc[match[1]:match[2]])

1
(451313080118390996, 47, 67) 'and what is the use of a book,' thought Alice 'without pictures or conversation?'


### 6.5.2. Problem with this Approach

In [36]:
for text in data[0][2]:
  text = text.replace("`", "'")
  doc = nlp(text)
  matches = matcher(doc)
  print(len(matches))
  matches.sort(key = lambda x: x[1])
  for match in matches[:20]:
    print(match, doc[match[1]:match[2]])

1
(451313080118390996, 47, 67) 'and what is the use of a book,' thought Alice 'without pictures or conversation?'
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


### 6.5.3. Adding More Patterns

In [37]:
speak_lemmas = ["think", "say"]
text = data[0][2][0].replace( "`", "'")
matcher = Matcher(nlp.vocab)
pattern1 = [{'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}, {"POS": "VERB", "LEMMA": {"IN": speak_lemmas}}, {"POS": "PROPN", "OP": "+"}, {'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}]
pattern2 = [{'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}, {"POS": "VERB", "LEMMA": {"IN": speak_lemmas}}, {"POS": "PROPN", "OP": "+"}]
pattern3 = [{"POS": "PROPN", "OP": "+"},{"POS": "VERB", "LEMMA": {"IN": speak_lemmas}}, {'ORTH': "'"}, {'IS_ALPHA': True, "OP": "+"}, {'IS_PUNCT': True, "OP": "*"}, {'ORTH': "'"}]
matcher.add("PROPER_NOUNS", [pattern1, pattern2, pattern3], greedy='LONGEST')
for text in data[0][2]:
    text = text.replace("`", "'")
    doc = nlp(text)
    matches = matcher(doc)
    matches.sort(key = lambda x: x[1])
    print (len(matches))
    for match in matches[:10]:
        print (match, doc[match[1]:match[2]])

1
(3232560085755078826, 47, 67) 'and what is the use of a book,' thought Alice 'without pictures or conversation?'
0
0
0
0
0
1
(3232560085755078826, 0, 6) 'Well!' thought Alice
0
0
0
0
0
0
0
1
(3232560085755078826, 57, 68) 'which certainly was not here before,' said Alice
0
0


# 7. Custom Components in spaCy

In [38]:
import spacy

In [39]:
doc = nlp("Britain is a place. Mary is a doctor.")

In [41]:
for ent in doc.ents:
  print(ent.text, ent.label_)

Britain GPE
Mary PERSON


In [42]:
from spacy.language import Language

In [43]:
@Language.component("remove_gpe")
def remove_gpe(doc):
  original_ents = list(doc.ents)
  for ent in doc.ents:
    if ent.label_ == "GPE":
      original_ents.remove(ent)
  doc.ents = original_ents
  return(doc)

In [44]:
nlp.add_pipe("remove_gpe")

<function __main__.remove_gpe(doc)>

In [45]:
nlp.analyze_pipes()

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'remove_gpe': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False}},
 'problems': {'tok2vec': [],
  

In [46]:
doc = nlp("Britain is a place. Mary is a doctor.")
for ent in doc.ents:
  print(ent.text, ent.label_)

Mary PERSON


In [49]:
nlp.to_disk("/content/new_en_core_web_sm")